In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [71.4 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:14 http://ppa.launchpad.net/gr

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-07 06:29:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.44MB/s    in 0.2s    

2023-02-07 06:29:59 (5.44 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [5]:
# 1. Filter the data and create a new DataFrame or table to retrieve all the rows where the 
# total_votes count is equal to or greater than 20 to pick reviews that are more likely to be 
# helpful and to avoid having division by zero errors later on.

vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vote_count = vine_df.filter("total_votes >= 20")
vote_count.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|3          |54           |59         |N   |Y                |
|R13VL62Y2HBQ0B|5          |15           |21         |N   |Y                |
|R22G55KAPZKJQV|4          |20           |21         |N   |Y                |
|R1610PGTJS7G3N|2          |28           |44         |N   |Y                |
|RLQL04BL0QXOJ |4          |45           |47         |N   |Y                |
|R2AYJHH8WJNGAU|4          |98           |104        |N   |Y                |
|R111DJA10Y6CMU|5          |26           |39         |N   |Y                |
|R2EE2TR4MRDV0U|5          |53           |59         |N   |Y                |
|RD4A80I5JDHED |5          |69           |77         |N   |N                |
|R1GU6IYZQWQE8X|2          |24           |25         |N   |Y    

In [6]:
# 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or 
# table to retrieve all the rows where the number of helpful_votes divided by total_votes 
# is equal to or greater than 50%.

cleaned_helpful_votes = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"] >=0.5)
cleaned_helpful_votes.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|3          |54           |59         |N   |Y                |
|R13VL62Y2HBQ0B|5          |15           |21         |N   |Y                |
|R22G55KAPZKJQV|4          |20           |21         |N   |Y                |
|R1610PGTJS7G3N|2          |28           |44         |N   |Y                |
|RLQL04BL0QXOJ |4          |45           |47         |N   |Y                |
|R2AYJHH8WJNGAU|4          |98           |104        |N   |Y                |
|R111DJA10Y6CMU|5          |26           |39         |N   |Y                |
|R2EE2TR4MRDV0U|5          |53           |59         |N   |Y                |
|RD4A80I5JDHED |5          |69           |77         |N   |N                |
|R1GU6IYZQWQE8X|2          |24           |25         |N   |Y    

In [7]:
# 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame 
# or table that retrieves all the rows where a review was written as part of the 
# Vine program (paid), vine == 'Y'.

vine_yes = cleaned_helpful_votes.filter(cleaned_helpful_votes['vine'] == 'Y')
vine_yes.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1MAOLI5FJHAFM|4          |249          |261        |Y   |N                |
|R9PYAUDIBJVEC |4          |12           |22         |Y   |N                |
|R6V9SHMMG5M8F |5          |101          |110        |Y   |N                |
|R37PVLT6ELL5J4|4          |181          |209        |Y   |N                |
|R2FSFGWZF24V9 |4          |50           |51         |Y   |N                |
|R3SRW1E8J56IGV|5          |262          |281        |Y   |N                |
|R86Z11D4CWOFM |4          |32           |36         |Y   |N                |
|RNP01HW9YISJO |4          |20           |23         |Y   |N                |
|R3KLACA6LCDZ0S|3          |21           |22         |Y   |N                |
|RZEQYOT2RE0N7 |4          |75           |80         |Y   |N    

In [8]:
# 4. Repeat Step 3, but this time retrieve all the rows where the review was 
# not part of the Vine program (unpaid), vine == 'N'.

vine_no = cleaned_helpful_votes.filter(cleaned_helpful_votes['vine'] == 'N')
vine_no.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|3          |54           |59         |N   |Y                |
|R13VL62Y2HBQ0B|5          |15           |21         |N   |Y                |
|R22G55KAPZKJQV|4          |20           |21         |N   |Y                |
|R1610PGTJS7G3N|2          |28           |44         |N   |Y                |
|RLQL04BL0QXOJ |4          |45           |47         |N   |Y                |
|R2AYJHH8WJNGAU|4          |98           |104        |N   |Y                |
|R111DJA10Y6CMU|5          |26           |39         |N   |Y                |
|R2EE2TR4MRDV0U|5          |53           |59         |N   |Y                |
|RD4A80I5JDHED |5          |69           |77         |N   |N                |
|R1GU6IYZQWQE8X|2          |24           |25         |N   |Y    

In [9]:
# 5. Determine the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

paid_reviews = vine_yes.count()
unpaid_reviews = vine_no.count()

paid_fivestar = vine_yes.filter(vine_yes['star_rating'] == 5).count()
unpaid_fivestar = vine_no.filter(vine_no['star_rating'] == 5).count()

paid_percentage_fivestar = (paid_fivestar/paid_reviews) * 100
unpaid_percentage_fivestar = (unpaid_fivestar/unpaid_reviews) * 100
#print("Accutacy of model at prediting reviews was: %f" % acc)

print("Total number of  paid reviews: %f" % paid_reviews)
print("Total number of paid 5 star reviews: %f" % paid_fivestar)
print("Percentage of  paid 5-star reviews: %f" % paid_percentage_fivestar)
print("Total number of  unpaid reviews: %f" % unpaid_reviews)
print("Total number of unpaid 5 star reviews: %f" % unpaid_fivestar)
print("Percentage of  unpaid 5-star reviews: %f" % unpaid_percentage_fivestar)

Total number of  paid reviews: 613.000000
Total number of paid 5 star reviews: 222.000000
Percentage of  paid 5-star reviews: 36.215334
Total number of  unpaid reviews: 64968.000000
Total number of unpaid 5 star reviews: 30543.000000
Percentage of  unpaid 5-star reviews: 47.012375
